In [1]:
import altair as alt
import pandas as pd
import torch
from torch_geometric.loader import DataLoader

from main_layered import create_and_train_model, eval_model
from utils.data import gen_rectangular_channel_matrix, flex_graph

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def train_multiple_models(layers):
    k = 32
    data = gen_rectangular_channel_matrix(k, k, 10000, seed=11)
    data = flex_graph(data)
    data = DataLoader(data, batch_size=64, shuffle=True)
    for i in layers:
        path_ = f"./experiments/flexible_experiment_{i}_layers.pth"
        create_and_train_model(n=10000, k=32, batch_size=64, noise_var=1., path=path_, data=data, lr=0.002, layers=i)

def evaluate_models(layers):
    perf = []
    k = 32
    data = gen_rectangular_channel_matrix(k, k, 100, seed=899)
    data = flex_graph(data)
    for i in layers:
        path_ = f"./experiments/flexible_experiment_{i}_layers.pth"
        perf.append(eval_model(path=path_, data=data, layers=i).item())
    return perf

In [4]:
layers = [1, 2, 3, 4, 5]
# train_multiple_models(layers)

In [5]:
perf_layers = evaluate_models(layers)

-3.4525604248046875
-3.677065134048462
-3.9491357803344727
-3.9616055488586426
-4.01860237121582


In [6]:
def evaluate_models(samples):
    perf = []
    k = 32
    data = gen_rectangular_channel_matrix(k, k, 10000, seed=899)
    data = flex_graph(data)
    for i in samples:
        path_ = f"./flexible_experiment_{i}_training_samples.pth"
        perf.append(eval_model(path=path_, data=data))
    return perf

In [8]:
source = pd.DataFrame({'Layers': layers,
                           'Perf': perf_layers
                           })
source = source.melt('Layers', var_name='category', value_name='Performance')

layers_chart = alt.Chart(source).transform_joinaggregate(
    max='max(Performance)',
).transform_calculate(
    percent="datum.Performance / datum.max"
).transform_filter(
    0.85 <= alt.datum.percent
).mark_area(
    color="lightblue",
    line=True,
    point=True,
    interpolate='monotone'
).encode(
    alt.X('Layers:Q', axis=alt.Axis(tickCount=5)),
    alt.Y('percent:Q', title='Performance',
          scale=alt.Scale(zero=False, nice=True, type='linear', 
                          domain=[0.85, 1], padding=0
                         ),
          axis=alt.Axis(tickCount=6, format='.0%')),
    color=alt.value('#D95F02')
).configure_point(
    size=50
)
layers_chart

Displaying chart at http://localhost:58486/


![svg image](data:image/svg+xml,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHhtbG5zOnhsaW5rPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5L3hsaW5rIiB2ZXJzaW9uPSIxLjEiIGNsYXNzPSJtYXJrcyIgd2lkdGg9IjQ2MiIgaGVpZ2h0PSIzNDciIHZpZXdCb3g9IjAgMCA0NjIgMzQ3Ij48cmVjdCB3aWR0aD0iNDYyIiBoZWlnaHQ9IjM0NyIgZmlsbD0id2hpdGUiLz48ZyBmaWxsPSJub25lIiBzdHJva2UtbWl0ZXJsaW1pdD0iMTAiIHRyYW5zZm9ybT0idHJhbnNsYXRlKDUzLDEwKSI%2BPGcgY2xhc3M9Im1hcmstZ3JvdXAgcm9sZS1mcmFtZSByb290IiByb2xlPSJncmFwaGljcy1vYmplY3QiIGFyaWEtcm9sZWRlc2NyaXB0aW9uPSJncm91cCBtYXJrIGNvbnRhaW5lciI%2BPGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwwKSI%2BPHBhdGggY2xhc3M9ImJhY2tncm91bmQiIGFyaWEtaGlkZGVuPSJ0cnVlIiBkPSJNMC41LDAuNWg0MDB2MzAwaC00MDBaIiBzdHJva2U9IiNkZGQiLz48Zz48ZyBjbGFzcz0ibWFyay1ncm91cCByb2xlLWF4aXMiIHJvbGU9ImdyYXBoaWNzLXN5bWJvbCIgYXJpYS1yb2xlZGVzY3JpcHRpb249ImF4aXMiIGFyaWEtbGFiZWw9IlgtYXhpcyBmb3IgYSBsaW5lYXIgc2NhbGUgd2l0aCB2YWx1ZXMgZnJvbSAxIHRvIDUiPjxnIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAuNSwzMDAuNSkiPjxwYXRoIGNsYXNzPSJiYWNrZ3JvdW5kIiBhcmlhLWhpZGRlbj0idHJ1ZSIgZD0iTTAsMGgwdjBoMFoiIHBvaW50ZXItZXZlbnRzPSJub25lIi8%2BPGc%2BPGcgY2xhc3M9Im1hcmstcnVsZSByb2xlLWF4aXMtZ3JpZCIgcG9pbnRlci1ldmVudHM9Im5vbmUiPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsLTMwMCkiIHgyPSIwIiB5Mj0iMzAwIiBzdHJva2U9IiNkZGQiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDEwMCwtMzAwKSIgeDI9IjAiIHkyPSIzMDAiIHN0cm9rZT0iI2RkZCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMjAwLC0zMDApIiB4Mj0iMCIgeTI9IjMwMCIgc3Ryb2tlPSIjZGRkIiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgzMDAsLTMwMCkiIHgyPSIwIiB5Mj0iMzAwIiBzdHJva2U9IiNkZGQiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDQwMCwtMzAwKSIgeDI9IjAiIHkyPSIzMDAiIHN0cm9rZT0iI2RkZCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPC9nPjwvZz48cGF0aCBjbGFzcz0iZm9yZWdyb3VuZCIgYXJpYS1oaWRkZW49InRydWUiIGQ9IiIgcG9pbnRlci1ldmVudHM9Im5vbmUiIGRpc3BsYXk9Im5vbmUiLz48L2c%2BPC9nPjxnIGNsYXNzPSJtYXJrLWdyb3VwIHJvbGUtYXhpcyIgcm9sZT0iZ3JhcGhpY3Mtc3ltYm9sIiBhcmlhLXJvbGVkZXNjcmlwdGlvbj0iYXhpcyIgYXJpYS1sYWJlbD0iWS1heGlzIGZvciBhIGxpbmVhciBzY2FsZSB3aXRoIHZhbHVlcyBmcm9tIDAuODQgdG8gMS4wMCI%2BPGcgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMC41LDAuNSkiPjxwYXRoIGNsYXNzPSJiYWNrZ3JvdW5kIiBhcmlhLWhpZGRlbj0idHJ1ZSIgZD0iTTAsMGgwdjBoMFoiIHBvaW50ZXItZXZlbnRzPSJub25lIi8%2BPGc%2BPGcgY2xhc3M9Im1hcmstcnVsZSByb2xlLWF4aXMtZ3JpZCIgcG9pbnRlci1ldmVudHM9Im5vbmUiPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMzAwKSIgeDI9IjQwMCIgeTI9IjAiIHN0cm9rZT0iI2RkZCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwyNjIpIiB4Mj0iNDAwIiB5Mj0iMCIgc3Ryb2tlPSIjZGRkIiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgwLDIyNSkiIHgyPSI0MDAiIHkyPSIwIiBzdHJva2U9IiNkZGQiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMTg3KSIgeDI9IjQwMCIgeTI9IjAiIHN0cm9rZT0iI2RkZCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwxNTApIiB4Mj0iNDAwIiB5Mj0iMCIgc3Ryb2tlPSIjZGRkIiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgwLDExMykiIHgyPSI0MDAiIHkyPSIwIiBzdHJva2U9IiNkZGQiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsNzUpIiB4Mj0iNDAwIiB5Mj0iMCIgc3Ryb2tlPSIjZGRkIiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgwLDM4KSIgeDI9IjQwMCIgeTI9IjAiIHN0cm9rZT0iI2RkZCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwwKSIgeDI9IjQwMCIgeTI9IjAiIHN0cm9rZT0iI2RkZCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPC9nPjwvZz48cGF0aCBjbGFzcz0iZm9yZWdyb3VuZCIgYXJpYS1oaWRkZW49InRydWUiIGQ9IiIgcG9pbnRlci1ldmVudHM9Im5vbmUiIGRpc3BsYXk9Im5vbmUiLz48L2c%2BPC9nPjxnIGNsYXNzPSJtYXJrLWdyb3VwIHJvbGUtYXhpcyIgcm9sZT0iZ3JhcGhpY3Mtc3ltYm9sIiBhcmlhLXJvbGVkZXNjcmlwdGlvbj0iYXhpcyIgYXJpYS1sYWJlbD0iWC1heGlzIHRpdGxlZCAnTGF5ZXJzJyBmb3IgYSBsaW5lYXIgc2NhbGUgd2l0aCB2YWx1ZXMgZnJvbSAxIHRvIDUiPjxnIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAuNSwzMDAuNSkiPjxwYXRoIGNsYXNzPSJiYWNrZ3JvdW5kIiBhcmlhLWhpZGRlbj0idHJ1ZSIgZD0iTTAsMGgwdjBoMFoiIHBvaW50ZXItZXZlbnRzPSJub25lIi8%2BPGc%2BPGcgY2xhc3M9Im1hcmstcnVsZSByb2xlLWF4aXMtdGljayIgcG9pbnRlci1ldmVudHM9Im5vbmUiPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMCkiIHgyPSIwIiB5Mj0iNSIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgxMDAsMCkiIHgyPSIwIiB5Mj0iNSIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgyMDAsMCkiIHgyPSIwIiB5Mj0iNSIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgzMDAsMCkiIHgyPSIwIiB5Mj0iNSIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSg0MDAsMCkiIHgyPSIwIiB5Mj0iNSIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48L2c%2BPGcgY2xhc3M9Im1hcmstdGV4dCByb2xlLWF4aXMtbGFiZWwiIHBvaW50ZXItZXZlbnRzPSJub25lIj48dGV4dCB0ZXh0LWFuY2hvcj0ic3RhcnQiIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMTUpIiBmb250LWZhbWlseT0ic2Fucy1zZXJpZiIgZm9udC1zaXplPSIxMHB4IiBmaWxsPSIjMDAwIiBvcGFjaXR5PSIxIj4xPC90ZXh0Pjx0ZXh0IHRleHQtYW5jaG9yPSJtaWRkbGUiIHRyYW5zZm9ybT0idHJhbnNsYXRlKDEwMCwxNSkiIGZvbnQtZmFtaWx5PSJzYW5zLXNlcmlmIiBmb250LXNpemU9IjEwcHgiIGZpbGw9IiMwMDAiIG9wYWNpdHk9IjEiPjI8L3RleHQ%2BPHRleHQgdGV4dC1hbmNob3I9Im1pZGRsZSIgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMjAwLDE1KSIgZm9udC1mYW1pbHk9InNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMTBweCIgZmlsbD0iIzAwMCIgb3BhY2l0eT0iMSI%2BMzwvdGV4dD48dGV4dCB0ZXh0LWFuY2hvcj0ibWlkZGxlIiB0cmFuc2Zvcm09InRyYW5zbGF0ZSgzMDAsMTUpIiBmb250LWZhbWlseT0ic2Fucy1zZXJpZiIgZm9udC1zaXplPSIxMHB4IiBmaWxsPSIjMDAwIiBvcGFjaXR5PSIxIj40PC90ZXh0Pjx0ZXh0IHRleHQtYW5jaG9yPSJlbmQiIHRyYW5zZm9ybT0idHJhbnNsYXRlKDQwMCwxNSkiIGZvbnQtZmFtaWx5PSJzYW5zLXNlcmlmIiBmb250LXNpemU9IjEwcHgiIGZpbGw9IiMwMDAiIG9wYWNpdHk9IjEiPjU8L3RleHQ%2BPC9nPjxnIGNsYXNzPSJtYXJrLXJ1bGUgcm9sZS1heGlzLWRvbWFpbiIgcG9pbnRlci1ldmVudHM9Im5vbmUiPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMCkiIHgyPSI0MDAiIHkyPSIwIiBzdHJva2U9IiM4ODgiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjwvZz48ZyBjbGFzcz0ibWFyay10ZXh0IHJvbGUtYXhpcy10aXRsZSIgcG9pbnRlci1ldmVudHM9Im5vbmUiPjx0ZXh0IHRleHQtYW5jaG9yPSJtaWRkbGUiIHRyYW5zZm9ybT0idHJhbnNsYXRlKDIwMCwzMCkiIGZvbnQtZmFtaWx5PSJzYW5zLXNlcmlmIiBmb250LXNpemU9IjExcHgiIGZvbnQtd2VpZ2h0PSJib2xkIiBmaWxsPSIjMDAwIiBvcGFjaXR5PSIxIj5MYXllcnM8L3RleHQ%2BPC9nPjwvZz48cGF0aCBjbGFzcz0iZm9yZWdyb3VuZCIgYXJpYS1oaWRkZW49InRydWUiIGQ9IiIgcG9pbnRlci1ldmVudHM9Im5vbmUiIGRpc3BsYXk9Im5vbmUiLz48L2c%2BPC9nPjxnIGNsYXNzPSJtYXJrLWdyb3VwIHJvbGUtYXhpcyIgcm9sZT0iZ3JhcGhpY3Mtc3ltYm9sIiBhcmlhLXJvbGVkZXNjcmlwdGlvbj0iYXhpcyIgYXJpYS1sYWJlbD0iWS1heGlzIHRpdGxlZCAnUGVyZm9ybWFuY2UnIGZvciBhIGxpbmVhciBzY2FsZSB3aXRoIHZhbHVlcyBmcm9tIDg0JSB0byAxMDAlIj48ZyB0cmFuc2Zvcm09InRyYW5zbGF0ZSgwLjUsMC41KSI%2BPHBhdGggY2xhc3M9ImJhY2tncm91bmQiIGFyaWEtaGlkZGVuPSJ0cnVlIiBkPSJNMCwwaDB2MGgwWiIgcG9pbnRlci1ldmVudHM9Im5vbmUiLz48Zz48ZyBjbGFzcz0ibWFyay1ydWxlIHJvbGUtYXhpcy10aWNrIiBwb2ludGVyLWV2ZW50cz0ibm9uZSI%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwzMDApIiB4Mj0iLTUiIHkyPSIwIiBzdHJva2U9IiM4ODgiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMjYyKSIgeDI9Ii01IiB5Mj0iMCIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgwLDIyNSkiIHgyPSItNSIgeTI9IjAiIHN0cm9rZT0iIzg4OCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwxODcpIiB4Mj0iLTUiIHkyPSIwIiBzdHJva2U9IiM4ODgiIHN0cm9rZS13aWR0aD0iMSIgb3BhY2l0eT0iMSIvPjxsaW5lIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMTUwKSIgeDI9Ii01IiB5Mj0iMCIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48bGluZSB0cmFuc2Zvcm09InRyYW5zbGF0ZSgwLDExMykiIHgyPSItNSIgeTI9IjAiIHN0cm9rZT0iIzg4OCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCw3NSkiIHgyPSItNSIgeTI9IjAiIHN0cm9rZT0iIzg4OCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwzOCkiIHgyPSItNSIgeTI9IjAiIHN0cm9rZT0iIzg4OCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwwKSIgeDI9Ii01IiB5Mj0iMCIgc3Ryb2tlPSIjODg4IiBzdHJva2Utd2lkdGg9IjEiIG9wYWNpdHk9IjEiLz48L2c%2BPGcgY2xhc3M9Im1hcmstdGV4dCByb2xlLWF4aXMtbGFiZWwiIHBvaW50ZXItZXZlbnRzPSJub25lIj48dGV4dCB0ZXh0LWFuY2hvcj0iZW5kIiB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtNywzMDMpIiBmb250LWZhbWlseT0ic2Fucy1zZXJpZiIgZm9udC1zaXplPSIxMHB4IiBmaWxsPSIjMDAwIiBvcGFjaXR5PSIxIj44NCU8L3RleHQ%2BPHRleHQgdGV4dC1hbmNob3I9ImVuZCIgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTcsMjY1LjQ5OTk5OTk5OTk5OTk0KSIgZm9udC1mYW1pbHk9InNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMTBweCIgZmlsbD0iIzAwMCIgb3BhY2l0eT0iMSI%2BODYlPC90ZXh0Pjx0ZXh0IHRleHQtYW5jaG9yPSJlbmQiIHRyYW5zZm9ybT0idHJhbnNsYXRlKC03LDIyNy45OTk5OTk5OTk5OTk5NCkiIGZvbnQtZmFtaWx5PSJzYW5zLXNlcmlmIiBmb250LXNpemU9IjEwcHgiIGZpbGw9IiMwMDAiIG9wYWNpdHk9IjEiPjg4JTwvdGV4dD48dGV4dCB0ZXh0LWFuY2hvcj0iZW5kIiB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtNywxOTAuNDk5OTk5OTk5OTk5OTQpIiBmb250LWZhbWlseT0ic2Fucy1zZXJpZiIgZm9udC1zaXplPSIxMHB4IiBmaWxsPSIjMDAwIiBvcGFjaXR5PSIxIj45MCU8L3RleHQ%2BPHRleHQgdGV4dC1hbmNob3I9ImVuZCIgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTcsMTUyLjk5OTk5OTk5OTk5OTkpIiBmb250LWZhbWlseT0ic2Fucy1zZXJpZiIgZm9udC1zaXplPSIxMHB4IiBmaWxsPSIjMDAwIiBvcGFjaXR5PSIxIj45MiU8L3RleHQ%2BPHRleHQgdGV4dC1hbmNob3I9ImVuZCIgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTcsMTE1LjUwMDAwMDAwMDAwMDA3KSIgZm9udC1mYW1pbHk9InNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMTBweCIgZmlsbD0iIzAwMCIgb3BhY2l0eT0iMSI%2BOTQlPC90ZXh0Pjx0ZXh0IHRleHQtYW5jaG9yPSJlbmQiIHRyYW5zZm9ybT0idHJhbnNsYXRlKC03LDc4LjAwMDAwMDAwMDAwMDA3KSIgZm9udC1mYW1pbHk9InNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMTBweCIgZmlsbD0iIzAwMCIgb3BhY2l0eT0iMSI%2BOTYlPC90ZXh0Pjx0ZXh0IHRleHQtYW5jaG9yPSJlbmQiIHRyYW5zZm9ybT0idHJhbnNsYXRlKC03LDQwLjUwMDAwMDAwMDAwMDAzNikiIGZvbnQtZmFtaWx5PSJzYW5zLXNlcmlmIiBmb250LXNpemU9IjEwcHgiIGZpbGw9IiMwMDAiIG9wYWNpdHk9IjEiPjk4JTwvdGV4dD48dGV4dCB0ZXh0LWFuY2hvcj0iZW5kIiB0cmFuc2Zvcm09InRyYW5zbGF0ZSgtNywzKSIgZm9udC1mYW1pbHk9InNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMTBweCIgZmlsbD0iIzAwMCIgb3BhY2l0eT0iMSI%2BMTAwJTwvdGV4dD48L2c%2BPGcgY2xhc3M9Im1hcmstcnVsZSByb2xlLWF4aXMtZG9tYWluIiBwb2ludGVyLWV2ZW50cz0ibm9uZSI%2BPGxpbmUgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMCwzMDApIiB4Mj0iMCIgeTI9Ii0zMDAiIHN0cm9rZT0iIzg4OCIgc3Ryb2tlLXdpZHRoPSIxIiBvcGFjaXR5PSIxIi8%2BPC9nPjxnIGNsYXNzPSJtYXJrLXRleHQgcm9sZS1heGlzLXRpdGxlIiBwb2ludGVyLWV2ZW50cz0ibm9uZSI%2BPHRleHQgdGV4dC1hbmNob3I9Im1pZGRsZSIgdHJhbnNmb3JtPSJ0cmFuc2xhdGUoLTM2LjU3NjE3MTg3NSwxNTApIHJvdGF0ZSgtOTApIHRyYW5zbGF0ZSgwLC0yKSIgZm9udC1mYW1pbHk9InNhbnMtc2VyaWYiIGZvbnQtc2l6ZT0iMTFweCIgZm9udC13ZWlnaHQ9ImJvbGQiIGZpbGw9IiMwMDAiIG9wYWNpdHk9IjEiPlBlcmZvcm1hbmNlPC90ZXh0PjwvZz48L2c%2BPHBhdGggY2xhc3M9ImZvcmVncm91bmQiIGFyaWEtaGlkZGVuPSJ0cnVlIiBkPSIiIHBvaW50ZXItZXZlbnRzPSJub25lIiBkaXNwbGF5PSJub25lIi8%2BPC9nPjwvZz48ZyBjbGFzcz0ibWFyay1hcmVhIHJvbGUtbWFyayBsYXllcl8wX21hcmtzIiByb2xlPSJncmFwaGljcy1zeW1ib2wiIGFyaWEtcm9sZWRlc2NyaXB0aW9uPSJhcmVhIG1hcmsgY29udGFpbmVyIj48cGF0aCBkPSJNMCwyNjQuMTAzOTIyNTc4Mzc5QzMzLjMzMzMzMzMzMzMzMzMzNiwyMzEuMDM2ODg3NDkwMTgxMDIsNjYuNjY2NjY2NjY2NjY2NjYsMTk3Ljk2OTg1MjQwMTk4MzAyLDEwMCwxNTkuMzU0NDg3MDkxMjU0NzVDMTMzLjMzMzMzMzMzMzMzMzM0LDEyMC43MzkxMjE3ODA1MjY0OSwxNjYuNjY2NjY2NjY2NjY2NjYsMzYuMjkwNDk0ODI1MzQ4MTgsMjAwLDMyLjQxMTczMDcxNDAwOTQ2NUMyMzMuMzMzMzMzMzMzMzMzMzQsMjguNTMyOTY2NjAyNjcwNzUsMjY2LjY2NjY2NjY2NjY2NjcsMzAuNDcyMzQ4NjU4MzQwMTA2LDMwMCwyNi41OTM1ODQ1NDcwMDEzOUMzMzMuMzMzMzMzMzMzMzMzMywyMi43MTQ4MjA0MzU2NjI2NzcsMzY2LjY2NjY2NjY2NjY2NjcsMTEuMzU3NDEwMjE3ODMxMzM3LDQwMCwwTDQwMCwzMDBDMzY2LjY2NjY2NjY2NjY2NjcsMzAwLDMzMy4zMzMzMzMzMzMzMzMzLDMwMCwzMDAsMzAwQzI2Ni42NjY2NjY2NjY2NjY3LDMwMCwyMzMuMzMzMzMzMzMzMzMzMzQsMzAwLDIwMCwzMDBDMTY2LjY2NjY2NjY2NjY2NjY2LDMwMCwxMzMuMzMzMzMzMzMzMzMzMzQsMzAwLDEwMCwzMDBDNjYuNjY2NjY2NjY2NjY2NjYsMzAwLDMzLjMzMzMzMzMzMzMzMzMzNiwzMDAsMCwzMDBaIiBmaWxsPSIjRDk1RjAyIiBvcGFjaXR5PSIwLjciLz48L2c%2BPGcgY2xhc3M9Im1hcmstbGluZSByb2xlLW1hcmsgbGF5ZXJfMV9tYXJrcyIgcm9sZT0iZ3JhcGhpY3Mtc3ltYm9sIiBhcmlhLXJvbGVkZXNjcmlwdGlvbj0ibGluZSBtYXJrIGNvbnRhaW5lciI%2BPHBhdGggZD0iTTAsMjY0LjEwMzkyMjU3ODM3OUMzMy4zMzMzMzMzMzMzMzMzMzYsMjMxLjAzNjg4NzQ5MDE4MTAyLDY2LjY2NjY2NjY2NjY2NjY2LDE5Ny45Njk4NTI0MDE5ODMwMiwxMDAsMTU5LjM1NDQ4NzA5MTI1NDc1QzEzMy4zMzMzMzMzMzMzMzMzNCwxMjAuNzM5MTIxNzgwNTI2NDksMTY2LjY2NjY2NjY2NjY2NjY2LDM2LjI5MDQ5NDgyNTM0ODE4LDIwMCwzMi40MTE3MzA3MTQwMDk0NjVDMjMzLjMzMzMzMzMzMzMzMzM0LDI4LjUzMjk2NjYwMjY3MDc1LDI2Ni42NjY2NjY2NjY2NjY3LDMwLjQ3MjM0ODY1ODM0MDEwNiwzMDAsMjYuNTkzNTg0NTQ3MDAxMzlDMzMzLjMzMzMzMzMzMzMzMzMsMjIuNzE0ODIwNDM1NjYyNjc3LDM2Ni42NjY2NjY2NjY2NjY3LDExLjM1NzQxMDIxNzgzMTMzNyw0MDAsMCIgc3Ryb2tlPSIjRDk1RjAyIiBzdHJva2Utd2lkdGg9IjIiLz48L2c%2BPGcgY2xhc3M9Im1hcmstc3ltYm9sIHJvbGUtbWFyayBsYXllcl8yX21hcmtzIiByb2xlPSJncmFwaGljcy1zeW1ib2wiIGFyaWEtcm9sZWRlc2NyaXB0aW9uPSJzeW1ib2wgbWFyayBjb250YWluZXIiPjxwYXRoIHRyYW5zZm9ybT0idHJhbnNsYXRlKDAsMjY0LjEwMzkyMjU3ODM3OSkiIGQ9Ik0zLjUzNTUzMzkwNTkzMjczNzgsMEEzLjUzNTUzMzkwNTkzMjczNzgsMy41MzU1MzM5MDU5MzI3Mzc4LDAsMSwxLC0zLjUzNTUzMzkwNTkzMjczNzgsMEEzLjUzNTUzMzkwNTkzMjczNzgsMy41MzU1MzM5MDU5MzI3Mzc4LDAsMSwxLDMuNTM1NTMzOTA1OTMyNzM3OCwwIiBmaWxsPSIjRDk1RjAyIiBzdHJva2Utd2lkdGg9IjIiIG9wYWNpdHk9IjEiLz48cGF0aCB0cmFuc2Zvcm09InRyYW5zbGF0ZSgxMDAsMTU5LjM1NDQ4NzA5MTI1NDc1KSIgZD0iTTMuNTM1NTMzOTA1OTMyNzM3OCwwQTMuNTM1NTMzOTA1OTMyNzM3OCwzLjUzNTUzMzkwNTkzMjczNzgsMCwxLDEsLTMuNTM1NTMzOTA1OTMyNzM3OCwwQTMuNTM1NTMzOTA1OTMyNzM3OCwzLjUzNTUzMzkwNTkzMjczNzgsMCwxLDEsMy41MzU1MzM5MDU5MzI3Mzc4LDAiIGZpbGw9IiNEOTVGMDIiIHN0cm9rZS13aWR0aD0iMiIgb3BhY2l0eT0iMSIvPjxwYXRoIHRyYW5zZm9ybT0idHJhbnNsYXRlKDIwMCwzMi40MTE3MzA3MTQwMDk0NjUpIiBkPSJNMy41MzU1MzM5MDU5MzI3Mzc4LDBBMy41MzU1MzM5MDU5MzI3Mzc4LDMuNTM1NTMzOTA1OTMyNzM3OCwwLDEsMSwtMy41MzU1MzM5MDU5MzI3Mzc4LDBBMy41MzU1MzM5MDU5MzI3Mzc4LDMuNTM1NTMzOTA1OTMyNzM3OCwwLDEsMSwzLjUzNTUzMzkwNTkzMjczNzgsMCIgZmlsbD0iI0Q5NUYwMiIgc3Ryb2tlLXdpZHRoPSIyIiBvcGFjaXR5PSIxIi8%2BPHBhdGggdHJhbnNmb3JtPSJ0cmFuc2xhdGUoMzAwLDI2LjU5MzU4NDU0NzAwMTM5KSIgZD0iTTMuNTM1NTMzOTA1OTMyNzM3OCwwQTMuNTM1NTMzOTA1OTMyNzM3OCwzLjUzNTUzMzkwNTkzMjczNzgsMCwxLDEsLTMuNTM1NTMzOTA1OTMyNzM3OCwwQTMuNTM1NTMzOTA1OTMyNzM3OCwzLjUzNTUzMzkwNTkzMjczNzgsMCwxLDEsMy41MzU1MzM5MDU5MzI3Mzc4LDAiIGZpbGw9IiNEOTVGMDIiIHN0cm9rZS13aWR0aD0iMiIgb3BhY2l0eT0iMSIvPjxwYXRoIHRyYW5zZm9ybT0idHJhbnNsYXRlKDQwMCwwKSIgZD0iTTMuNTM1NTMzOTA1OTMyNzM3OCwwQTMuNTM1NTMzOTA1OTMyNzM3OCwzLjUzNTUzMzkwNTkzMjczNzgsMCwxLDEsLTMuNTM1NTMzOTA1OTMyNzM3OCwwQTMuNTM1NTMzOTA1OTMyNzM3OCwzLjUzNTUzMzkwNTkzMjczNzgsMCwxLDEsMy41MzU1MzM5MDU5MzI3Mzc4LDAiIGZpbGw9IiNEOTVGMDIiIHN0cm9rZS13aWR0aD0iMiIgb3BhY2l0eT0iMSIvPjwvZz48L2c%2BPHBhdGggY2xhc3M9ImZvcmVncm91bmQiIGFyaWEtaGlkZGVuPSJ0cnVlIiBkPSIiIGRpc3BsYXk9Im5vbmUiLz48L2c%2BPC9nPjwvZz48L3N2Zz4%3D)